In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



Reading in NHPI csv

In [35]:
NHPI = pd.read_csv('NHPI.csv')

In [36]:
NHPI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63480 entries, 0 to 63479
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   REF_DATE                   63480 non-null  object 
 1   GEO                        63480 non-null  object 
 2   DGUID                      61893 non-null  object 
 3   New housing price indexes  63480 non-null  object 
 4   UOM                        63480 non-null  object 
 5   UOM_ID                     63480 non-null  int64  
 6   SCALAR_FACTOR              63480 non-null  object 
 7   SCALAR_ID                  63480 non-null  int64  
 8   VECTOR                     63480 non-null  object 
 9   COORDINATE                 63480 non-null  float64
 10  VALUE                      52654 non-null  float64
 11  STATUS                     28652 non-null  object 
 12  SYMBOL                     0 non-null      float64
 13  TERMINATED                 0 non-null      flo

In [37]:
NHPI.head()

,REF_DATE,GEO,DGUID,New housing price indexes,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,Jan-81,Canada,2016A000011124,Total (house and land),"Index, 201612=100",347,units,0,v111955442,1.1,38.2,NaN,NaN,NaN,1
1,Jan-81,Canada,2016A000011124,House only,"Index, 201612=100",347,units,0,v111955443,1.2,36.1,NaN,NaN,NaN,1
2,Jan-81,Canada,2016A000011124,Land only,"Index, 201612=100",347,units,0,v111955444,1.3,40.6,E,NaN,NaN,1
3,Jan-81,Atlantic Region,2016A00011,Total (house and land),"Index, 201612=100",347,units,0,v111955445,2.1,NaN,..,NaN,NaN,1
4,Jan-81,Atlantic Region,2016A00011,House only,"Index, 201612=100",347,units,0,v111955446,2.2,NaN,..,NaN,NaN,1


In [38]:
NHPI["REF_DATE"] = pd.to_datetime(NHPI["REF_DATE"], format="%b-%y")

In [39]:
NHPI_2= NHPI[["REF_DATE","GEO","New housing price indexes","VALUE"]]

In [40]:
NHPI_2.head()

,REF_DATE,GEO,New housing price indexes,VALUE
0,1981-01-01,Canada,Total (house and land),38.2
1,1981-01-01,Canada,House only,36.1
2,1981-01-01,Canada,Land only,40.6
3,1981-01-01,Atlantic Region,Total (house and land),NaN
4,1981-01-01,Atlantic Region,House only,NaN


In [41]:
NHPI_2

,REF_DATE,GEO,New housing price indexes,VALUE
0,1981-01-01,Canada,Total (house and land),38.2
1,1981-01-01,Canada,House only,36.1
2,1981-01-01,Canada,Land only,40.6
3,1981-01-01,Atlantic Region,Total (house and land),NaN
4,1981-01-01,Atlantic Region,House only,NaN
...,...,...,...,...
63475,2025-01-01,"Vancouver, British Columbia",House only,125.4
63476,2025-01-01,"Vancouver, British Columbia",Land only,123.8
63477,2025-01-01,"Victoria, British Columbia",Total (house and land),119.1
63478,2025-01-01,"Victoria, British Columbia",House only,122.6


In [42]:
NHPI_2= NHPI_2.pivot(index=['REF_DATE', 'GEO'], columns="New housing price indexes", values='VALUE')

In [43]:
NHPI_2.reset_index(inplace=True)

In [44]:
NHPI_2.columns.name = None

In [45]:
NHPI_2.rename(columns={'House only': 'House only NHPI', 'Land only': 'Land only NHPI', 'Total (house and land)': 'Total (house and land) NHPI'}, inplace=True)
# renaming each column to make sure we identify the NHPI later when merges happen

In [46]:
NHPI_2.shape

(21160, 5)

In [47]:
geo_unique_values = NHPI_2["GEO"].unique().tolist()

In [48]:
geo_unique_values

['Alberta',
 'Atlantic Region',
 'British Columbia',
 'Calgary, Alberta',
 'Canada',
 'Charlottetown, Prince Edward Island',
 'Edmonton, Alberta',
 'Greater Sudbury, Ontario',
 'Guelph, Ontario',
 'Halifax, Nova Scotia',
 'Hamilton, Ontario',
 'Kelowna, British Columbia',
 'Kitchener-Cambridge-Waterloo, Ontario',
 'London, Ontario',
 'Manitoba',
 'Montréal, Quebec',
 'New Brunswick',
 'Newfoundland and Labrador',
 'Nova Scotia',
 'Ontario',
 'Oshawa, Ontario',
 'Ottawa-Gatineau, Ontario part, Ontario/Quebec',
 'Ottawa-Gatineau, Quebec part, Ontario/Quebec',
 'Prairie Region',
 'Prince Edward Island',
 'Quebec',
 'Québec, Quebec',
 'Regina, Saskatchewan',
 'Saint John, Fredericton, and Moncton, New Brunswick',
 'Saskatchewan',
 'Saskatoon, Saskatchewan',
 'Sherbrooke, Quebec',
 'St. Catharines-Niagara, Ontario',
 "St. John's, Newfoundland and Labrador",
 'Toronto, Ontario',
 'Trois-Rivières, Quebec',
 'Vancouver, British Columbia',
 'Victoria, British Columbia',
 'Windsor, Ontario',
 'W

Preparing the dataset for merge, that is why we filter only the filters from GEO columb to regions that exisit on other dastasets


In [49]:
filtered_NHPI = NHPI_2[NHPI_2["GEO"].isin(["Alberta", "Canada","British Columbia","Manitoba","New Brunswick","Newfoundland and Labrador","Nova Scotia","Ontario","Prince Edward Island","Quebec","Saskatchewan"])]

In [33]:
filtered_NHPI.head()

,REF_DATE,GEO,House only NHPI,Land only NHPI,Total (house and land) NHPI
0,1981-01-01,Alberta,NaN,NaN,NaN
2,1981-01-01,British Columbia,NaN,NaN,NaN
4,1981-01-01,Canada,36.1,40.6,38.2
14,1981-01-01,Manitoba,NaN,NaN,NaN
16,1981-01-01,New Brunswick,NaN,NaN,NaN


In [ ]:
filtered_NHPI.to_csv("NHPI_Manipulated.csv", index=False)